# Proyecto 2 modelos

In [1]:
import pandas as pd
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import os
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import ndimage
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import cv2
from PIL import Image
from zipfile import ZipFile
from torchvision import transforms
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.decomposition import PCA

In [2]:
sample = 600

In [3]:
data_VGG = pd.read_csv('train_filtrado.csv')
data_VGG = data_VGG.sample(n=sample, random_state=42)

In [4]:
data_vertebrae = pd.read_csv('meta_train_with_vertebrae.csv')

In [5]:
data_VGG.head()

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
1125,1.2.826.0.1.3680043.30177,1,0,0,1,0,0,1,1
1030,1.2.826.0.1.3680043.5420,0,0,0,0,0,0,0,0
1683,1.2.826.0.1.3680043.32030,0,0,0,0,0,0,0,0
1541,1.2.826.0.1.3680043.19618,1,0,1,0,0,0,0,0
344,1.2.826.0.1.3680043.5476,1,1,0,0,0,0,0,0


In [6]:
data_vertebrae.head()

,StudyInstanceUID,Slice,ImageHeight,ImageWidth,SliceThickness,ImagePositionPatient_x,ImagePositionPatient_y,ImagePositionPatient_z,SliceRatio,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.10001,1,512,512,0.625,-52.308,-27.712,7.282,0.003731,0,0,0,0,0,0,0
1,1.2.826.0.1.3680043.10001,2,512,512,0.625,-52.308,-27.712,6.657,0.007463,0,0,0,0,0,0,0
2,1.2.826.0.1.3680043.10001,3,512,512,0.625,-52.308,-27.712,6.032,0.011194,0,0,0,0,0,0,0
3,1.2.826.0.1.3680043.10001,4,512,512,0.625,-52.308,-27.712,5.407,0.014925,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.10001,5,512,512,0.625,-52.308,-27.712,4.782,0.018657,0,0,0,0,0,0,0


In [7]:
# Parámetros del modelo
vgg_output_size = 1280  # Tamaño de la salida de la VGG16
gru_hidden_size = 128  # Tamaño del estado oculto de la GRU
gru_num_layers = 2  # Número de capas en la GRU
num_classes = 7  # Reemplaza con el número de clases en tu problema

In [8]:
class WeightedMultiLabelLogLoss(nn.Module):
    def __init__(self, weight=None):
        super(WeightedMultiLabelLogLoss, self).__init__()
        self.weight = weight

    def forward(self, input, target):
        """
        Computes the weighted multi-label logarithmic loss.

        Args:
            input (torch.Tensor): Predicted probabilities (output of the model).
                                Shape: (batch_size, num_classes)
            target (torch.Tensor): Target labels (ground truth).
                                Shape: (batch_size, num_classes)

        Returns:
            loss (torch.Tensor): Weighted multi-label logarithmic loss.
        """
        epsilon = 1e-15  # Small constant to avoid log(0)
        # Log loss
        log_loss = -target * torch.log(input + epsilon) - (1 - target) * torch.log(1 - input + epsilon)
        
        # Apply weights if provided
        if self.weight is not None:
            log_loss = log_loss * self.weight

        # Compute mean loss over samples and classes
        loss = log_loss.mean()
        return loss

In [9]:
# import torch
# import torch.nn as nn
# import torchvision.models as models

# # Define la arquitectura del modelo combinado
# class CombinedModel(nn.Module):
#     def __init__(self, vgg_output_size, gru_hidden_size, gru_num_layers, num_classes):
#         super(CombinedModel, self).__init__()
#         self.vgg16 = models.vgg16(pretrained=True)
#         self.vgg16.features[28] = nn.AvgPool2d(kernel_size=7, stride=1)  # Cambia el stride de la última capa de pooling
#         self.vgg16.classifier = nn.Identity()  # Desactiva las capas clasificadoras

#         self.gru = nn.GRU(vgg_output_size, gru_hidden_size, num_layers=gru_num_layers, batch_first=True)
#         self.fc = nn.Sequential(
#             nn.Linear(gru_hidden_size, 512),
#             nn.ReLU(),
#             nn.Dropout(p=0.5),  # Agrega dropout
#             nn.Linear(512, num_classes)
#         )


#     def forward(self, x):
#         # Pasa cada imagen a través de VGG16 y obtén los feature maps
#         vgg_outputs = []
#         for image in x:
#             vgg_output = self.vgg16(image)
#             vgg_output = vgg_output.view(vgg_output.size(0), -1, vgg_output.size(1))
#             vgg_outputs.append(vgg_output)

#         # Concatena los feature maps de las imágenes a lo largo de la dimensión del tiempo (secuencia)
#         gru_input = torch.cat(vgg_outputs, dim=0)  # Concatena en la dimensión 0 (batch_size x num_images, 512, height, width)

#         # Reorganiza la entrada para que cada imagen sea una "secuencia" para la GRU
#         gru_input = gru_input.view(-1, len(vgg_outputs), vgg_output_size)  # (batch_size, num_images, 512)

#         # Pasa la secuencia de entrada a través de GRU
#         gru_output, _ = self.gru(gru_input)

#         # Pasa la última salida de la GRU a través de una capa completamente conectada
#         output = self.fc(gru_output[:, -1, :])
#         return output


In [10]:
# class CombinedModel(nn.Module):
#     def __init__(self, resnet_output_size, gru_hidden_size, gru_num_layers, num_classes):
#         super(CombinedModel, self).__init__()
#         self.resnet18 = models.resnet18(pretrained=True)
#         # Elimina la capa de clasificación completamente conectada (fully connected layer)
#         self.resnet18 = nn.Sequential(*list(self.resnet18.children())[:-1])

#         self.gru = nn.GRU(resnet_output_size, gru_hidden_size, num_layers=gru_num_layers, batch_first=True)
#         self.fc = nn.Sequential(
#             nn.Linear(gru_hidden_size, 512),
#             nn.ReLU(),
#             nn.Dropout(p=0.5),  # Agrega dropout
#             nn.Linear(512, num_classes)
#         )

#     def forward(self, x):
#         # Pasa cada imagen a través de ResNet-18 y obtén los feature maps
#         resnet_outputs = []
#         for image in x:
#             resnet_output = self.resnet18(image)
#             resnet_output = resnet_output.view(resnet_output.size(0), -1, resnet_output.size(1))
#             resnet_outputs.append(resnet_output)

#         # Concatena los feature maps de las imágenes a lo largo de la dimensión del tiempo (secuencia)
#         gru_input = torch.cat(resnet_outputs, dim=0)  # Concatena en la dimensión 0 (batch_size x num_images, 512, 1, 1)

#         # Reorganiza la entrada para que cada imagen sea una "secuencia" para la GRU
#         gru_input = gru_input.squeeze(-1).squeeze(-1)  # Elimina dimensiones 2 y 3 (altura y ancho)
#         gru_input = gru_input.view(-1, len(resnet_outputs), gru_input.size(1))  # (batch_size, num_images, 512)
#         print(gru_input.shape)
#         # Pasa la secuencia de entrada a través de GRU
#         gru_output, _ = self.gru(gru_input)

#         # Pasa la última salida de la GRU a través de una capa completamente conectada
#         output = self.fc(gru_output[:, -1, :])
#         return output

In [11]:
# class CombinedModel(nn.Module):
#     def __init__(self, efficientnet_output_size, gru_hidden_size, gru_num_layers, num_classes):
#         super(CombinedModel, self).__init__()
#         self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')  # Puedes elegir otra versión de EfficientNet
#         self.efficientnet._fc = nn.Identity()  # Desactiva las capas clasificadoras

#         self.gru = nn.GRU(efficientnet_output_size, gru_hidden_size, num_layers=gru_num_layers, batch_first=True)
#         self.fc = nn.Sequential(
#             nn.Linear(gru_hidden_size, gru_hidden_size),
#             nn.ReLU(),
#             nn.Dropout(p=0.5),
#             nn.Linear(gru_hidden_size, num_classes)
#         )

#     def forward(self, x):
#         # Aplica EfficientNet a todas las imágenes de entrada
#         efficientnet_outputs = self.efficientnet.extract_features(x)

#         # Reformatea la salida de EfficientNet para que sea compatible con la GRU
#         efficientnet_outputs = efficientnet_outputs.view(efficientnet_outputs.size(0), -1, efficientnet_outputs.size(1))

#         # Pasa la salida de EfficientNet a través de la GRU
#         gru_output, _ = self.gru(efficientnet_outputs)

#         gru_output_avg = torch.mean(gru_output, dim=0)
        
#         # Aplica la capa completamente conectada
#         fc_output = self.fc(gru_output_avg)

#         # Aplica softmax para obtener la probabilidad para cada clase
#         output = F.softmax(fc_output, dim=1)
#         return output


In [12]:
import torch.nn.functional as F

class CombinedModel(nn.Module):
    def __init__(self, efficientnet_output_size, gru_hidden_size, gru_num_layers, num_classes):
        super(CombinedModel, self).__init__()
        
        # Cargar EfficientNet preentrenado
        self.efficientnet = models.efficientnet_b0(pretrained=True)
        # Eliminar la capa Fully Connected
        self.features = nn.Sequential(*list(self.efficientnet.children())[:-1])
        
        # Pooling Global Promedio
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        
        # GRU Layer
        self.gru = nn.GRU(input_size=efficientnet_output_size, hidden_size=gru_hidden_size, batch_first=True)
        
        # Fully Connected Layer
        self.fc = nn.Linear(gru_hidden_size, 1)
        self.dropout = nn.Dropout(0.5)
        
        # Softmax
        # self.softmax = nn.Softmax(dim=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        batch_size, timesteps, C, H, W = x.size()
        x = x.view(batch_size * timesteps, C, H, W)
        
        # Pasar imágenes por EfficientNet
        x = self.features(x)
        
        # Pooling Global Promedio
        x = self.global_avg_pool(x).squeeze(-1).squeeze(-1)
        
        x = x.view(batch_size, timesteps, -1)
        # print(x.shape)
        # Pasar la secuencia de feature maps por la GRU
        out, _ = self.gru(x)
        # Usar solo la última salida de la secuencia
        out = out[:, -1, :]
        # Pasar por la capa Fully Connected
        out = self.dropout(self.fc(out))
        # Softmax
        out = self.sigmoid(out)
        
        return out


In [13]:
if torch.cuda.is_available():
    print('GPU está disponible')
else:
    print('No se encontró GPU, usando CPU')

GPU está disponible


In [14]:
# Crea una instancia del modelo combinado
combined_model = CombinedModel(vgg_output_size, gru_hidden_size, gru_num_layers, num_classes)

c:\Users\Daniel\Main\UVG\Semestre VIII\Data science\Proyecto2\myenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Daniel\Main\UVG\Semestre VIII\Data science\Proyecto2\myenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
# # Cargar el modelo EfficientNet preentrenado (por ejemplo, EfficientNet-B0)
# efficientnet_model = EfficientNet.from_pretrained('efficientnet-b0')

# # Cambiar el modelo combinado para usar EfficientNet
# combined_model = CombinedModel(efficientnet_model, gru_hidden_size,
#                                 gru_num_layers, num_classes)

In [16]:
combined_model.to('cuda')

CombinedModel(
  (efficientnet): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (

In [17]:
# Define la función para preprocesar las imágenes para VGG16 y CombinedModel
def preprocess_image_for_combined_model(image):
    # Transformaciones para preprocesar las imágenes para VGG16
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Cambiar el tamaño a 224x224 (tamaño de entrada de la VGG16)
        transforms.ToTensor(),  # Convertir a tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalización requerida por VGG16
    ])

    preprocessed_image = transform(image)  # Aplica las transformaciones
    return preprocessed_image

In [18]:
# def save_images_to_npy(ct_images, labels, ct_name, save_dir='image_train_npy'):
#     os.makedirs(save_dir, exist_ok=True)
#     save_path = os.path.join(save_dir, ct_name + '.npz')
#     np.savez(save_path, images=ct_images, labels=labels)

In [19]:

# for current_idx in range (len(data_VGG)):
#     ct_name = data_VGG.iloc[current_idx]['StudyInstanceUID']
#     labels = data_VGG.iloc[current_idx][['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']].tolist()

#     with ZipFile(os.path.join('imagenes_train', ct_name + '.zip'), 'r') as zip_ref:
#         image_files = zip_ref.namelist()

#         # Read and process each image
#         ct_images = []
#         for image_file in image_files:
#             with zip_ref.open(image_file) as img_file:
#                 image = Image.open(img_file)  # Load the image
#                 image = preprocess_image_for_combined_model(image)
#                 ct_images.append(image)
#         save_images_to_npy(np.array(ct_images), np.array(labels), ct_name)

In [20]:
def apply_PCA(images):
    images_flat = images.reshape(images.shape[0], -1).numpy()  # Aplanar y convertir a numpy
    pca = PCA(n_components=70)  # Instanciar PCA
    transformed = pca.fit_transform(images_flat)  # Aplicar PCA
    return transformed

In [21]:
class ImageDataGenerator:
    def __init__(self, df, ct_folder, max_samples, cervical, vertebrae_info, batch_size=32):
        self.df = df
        self.ct_folder = ct_folder
        self.batch_size = batch_size
        self.num_samples = len(df)
        self.current_idx = 0
        self.cervical = cervical
        self.vertebrae_info = vertebrae_info
        self.max_samples = max_samples

    def __iter__(self):
        return self

    def __next__(self):
        batch_images = []
        batch_labels = []
        max_length = self.max_samples
        for _ in range(self.batch_size):
            if self.current_idx >= self.num_samples:
                self.current_idx = 0
                raise StopIteration

            ct_name = self.df.iloc[self.current_idx]['StudyInstanceUID']
            labels = [float(self.df.iloc[self.current_idx]['C' + str(self.cervical)])]
            filas_seleccionadas = self.vertebrae_info.loc[(self.vertebrae_info['StudyInstanceUID'] == ct_name) & (self.vertebrae_info['C' + str(self.cervical)] == 1)]
            # labels = self.df.iloc[self.current_idx][['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']].tolist()
            if not filas_seleccionadas.empty:
                slice_numbers = filas_seleccionadas['Slice'].tolist()
                with ZipFile(os.path.join(self.ct_folder, ct_name + '.zip'), 'r') as zip_ref:
                    image_files = zip_ref.namelist()

                    # Read and process each image
                    ct_images = []
                    counter = 0
                    for image_file in image_files:
                        slice_number = int(image_file.split('.jpg')[0].split('/')[-1])
                        if slice_number in slice_numbers:
                            with zip_ref.open(image_file) as img_file:
                                image = Image.open(img_file)  # Load the image
                                image = preprocess_image_for_combined_model(image)
                                ct_images.append(image)
                            counter += 1
                            if counter == max_length:
                                break
                    # ct_images = apply_PCA(torch.stack(ct_images))
                    # print(ct_images.shape)
                    # while counter < max_length:
                    #     ct_images.append(torch.zeros(3, 224, 224))
                    #     counter += 1
                    # Append images and labels to the batch
                    batch_images.append(ct_images)
                    batch_labels.append(labels)

            self.current_idx += 1

        
        return np.array(batch_images), np.array(batch_labels)
    
    def __len__(self):
        # Devuelve la longitud total del generador
        return self.num_samples

In [22]:
batch_size = 1

In [23]:
data_generator = ImageDataGenerator(data_VGG, 'imagenes_train', 100, 1, data_vertebrae, batch_size=batch_size)
# df, ct_folder, max_samples, cervical, vertebrae_info, batch_size=32)

In [24]:
# Especifica la carpeta donde se almacenarán los registros de TensorBoard
log_dir = "logs"

# Inicializa TensorBoard
writer = SummaryWriter(log_dir=os.path.join(log_dir, 'loss'))
accuracy_writer = SummaryWriter(log_dir=os.path.join(log_dir, 'accuracy'))

In [25]:
# Hiperparámetros de entrenamiento
learning_rate = 0.001
num_epochs = 20  # Número de épocas (iteraciones completas sobre el conjunto de datos)
display_step = 8
accumulation_steps = 4
# Suponiendo que `data_generator` es la instancia del generador de datos que creamos antes

# Definir la función de pérdida y el optimizador
# criterion = WeightedMultiLabelLogLoss()  # Suponiendo una tarea de clasificación
criterion = nn.BCELoss()
optimizer = optim.Adam(combined_model.parameters(), lr=learning_rate)

# Entrenamiento del modelo
combined_model.train()  # Poner el modelo en modo de entrenamiento
loss_acum = []

for epoch in range(num_epochs):
    total_loss = 0.0
    total_accuracy = 0.0
    running_loss = 0.0
    counter = 0
    running_accuracy = 0.0
    optimizer.zero_grad()  # Reiniciar los gradientes acumulados
    # Iterar a través del generador en mini lotes
    for i, (images_batch, labels_batch) in tqdm(enumerate(data_generator), total=sample):
    # for i, (images_batch, labels_batch) in enumerate(data_generator):
        # print(i)
        # Convertir el lote de imágenes a tensores y pasarlos por el modelo
        try:
            torch.cuda.empty_cache()
            images_tensor = torch.tensor(images_batch, dtype=torch.float32).to('cuda')
            labels_tensor = torch.tensor(labels_batch, dtype=torch.float32).to('cuda')  # Usar torch.long para las etiquetas
            # Realizar la propagación hacia adelante (forward pass)
            predictions = combined_model(images_tensor)
            # print(predictions, labels_batch)
            # predictions = torch.mean(predictions, dim=0, keepdim=False)
            loss = criterion(predictions, labels_tensor)
            threshold = 0.5
            predicted_labels = (predictions > threshold).float()  # 1 si es mayor al umbral, 0 de lo contrario
            predicted_labels_np = predicted_labels.cpu().numpy()
        
            # print(predictions)
            # Calcular la precisión
            correct_batch = (predicted_labels == labels_tensor).all(dim=1).sum().item()
            total_accuracy += correct_batch
            running_accuracy += correct_batch
            
            
            if (i + 1) % accumulation_steps == 0 or i == len(data_generator) - 1:
                loss.backward()
                optimizer.step()  # Actualizar los pesos
                

            # Acumular la pérdida total
            total_loss += loss.item()
            running_loss += loss.item()
            # total_accuracy += accuracy
            
            # Actualizar la pérdida en tiempo real en TensorBoard
            if i % display_step == display_step - 1:
                avg_loss = running_loss / display_step
                writer.add_scalar('Loss', avg_loss, epoch * len(data_generator) + i)

                running_loss = 0.0

            # Actualizar la precisión en tiempo real en TensorBoard
            if i % display_step == display_step - 1:
                avg_accuracy = running_accuracy / display_step
                accuracy_writer.add_scalar('Accuracy', avg_accuracy, epoch * len(data_generator) + i)
                running_accuracy = 0.0
        except:
            continue
        
        # if i % display_step == display_step - 1:
        #     # print('entre')
        #     avg_loss = running_loss / display_step
        #     writer.add_scalar('Loss', avg_loss, epoch * len(data_generator) + i)
        #     running_loss = 0.0
        counter += 1
        # if counter == 100:
        #     break
    # Calcular la pérdida promedio para la época
    average_loss = total_loss / sample
    loss_acum.append(average_loss)
    writer.flush()
    writer.close()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss:.4f}')
    print(f'Accuracy: {total_accuracy / sample:.4f}')

print('Entrenamiento completado.')

100%|██████████| 600/600 [16:17<00:00,  1.63s/it] 


Epoch [1/20], Loss: 0.6399
Accuracy: 0.8800


  8%|▊         | 49/600 [01:15<43:48,  4.77s/it]

In [ ]:
# Supongamos que ya has entrenado el modelo y deseas guardarlo
torch.save(combined_model.state_dict(), 'modelo_entrenado.pth')

In [ ]:
# Cargar el modelo entrenado
combined_model = CombinedModel(vgg_output_size, gru_hidden_size, gru_num_layers, num_classes)
combined_model.load_state_dict(torch.load('modelo_entrenado.pth'))